# BASIC

## 경로 설정

In [4]:
from pathlib import Path
import pandas as pd

# Path 정리
DATA_DIR = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/data")
BASE_DIR = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/ASAN_01_mini_sam3/")
CSV_PATH = DATA_DIR / "metadata.csv"

# CSV 불러오기
df = pd.read_csv(CSV_PATH)

target = 0                             # 원하는 행 인덱스 설정
COMMON_PATH = df.loc[target,"common_path"]   # COMMON_PATH 추출

VIDEO_PTH = df.loc[target,"video_path"]
N_FRAMES = df.loc[target,"n_frames"]                    # 프레임 수 추출

FRAME_DIR = DATA_DIR / "1_FRAME" / COMMON_PATH          # 프레임 디렉토리 경로 설정
KPT_DIR = DATA_DIR / "2_KEYPOINTS" / COMMON_PATH        # 키포인트 디렉토리 경로 설정
MP4_PATH = DATA_DIR / "3_MP4" / f"{COMMON_PATH}.mp4"     # MP4 디렉토리 경로 설정    
INTERP_DIR = DATA_DIR / "4_INTERP_DATA" /COMMON_PATH    # 보간 데이터 디렉토리 경로 설정
OUTPUT_PATH = DATA_DIR / "test"

CHECKPOINT_DIR = DATA_DIR / "checkpoints/SAM3"
CHECKPOINT_PT = CHECKPOINT_DIR / "sam3.pt"

print(COMMON_PATH)

### 디버깅

In [5]:
import sys

print("\n" + "="*60)
print("🐞 DEBUGGING REPORT")
print("="*60)

# 1. 기본 경로 및 CSV 확인
print(f"\n[1] 기본 경로 설정 확인")
print(f"   - DATA_DIR: {DATA_DIR} [{'✅ Exists' if DATA_DIR.exists() else '❌ Missing'}]")
print(f"   - BASE_DIR: {BASE_DIR} [{'✅ Exists' if BASE_DIR.exists() else '❌ Missing'}]")
print(f"   - CSV_PATH: {CSV_PATH} [{'✅ Exists' if CSV_PATH.exists() else '❌ Missing'}]")

# 2. 타겟 데이터 정보 (DataFrame 추출 값)
print(f"\n[2] Target Data Info (Index: {target})")
print(f"   - COMMON_PATH: {COMMON_PATH}")
print(f"   - N_FRAMES   : {N_FRAMES} (Type: {type(N_FRAMES).__name__})")
print(f"   - VIDEO_PTH (Raw): {VIDEO_PTH}")

# 3. 주요 디렉토리 및 파일 존재 여부 검증
# VIDEO_PTH는 문자열일 수 있으므로 Path 객체로 변환하여 확인
video_path_obj = Path(VIDEO_PTH)
print(f"\n[3] 주요 경로 유효성 검사")
print(f"   - Video File   : {video_path_obj} [{'✅ Exists' if video_path_obj.exists() else '❌ Missing'}]")
print(f"   - MP4_PATH     : {MP4_PATH} [{'✅ Exists' if MP4_PATH.exists() else '❌ Missing'}]")
print(f"   - FRAME_DIR    : {FRAME_DIR} [{'✅ Exists' if FRAME_DIR.exists() else '❌ Missing'}]")
print(f"   - KPT_DIR      : {KPT_DIR} [{'✅ Exists' if KPT_DIR.exists() else '❌ Missing'}]")
print(f"   - INTERP_DIR   : {INTERP_DIR} [{'✅ Exists' if INTERP_DIR.exists() else '❌ Missing'}]")
print(f"   - OUTPUT_PATH  : {OUTPUT_PATH} [{'✅ Exists' if OUTPUT_PATH.exists() else '⚠️ Will be created'}]")

# 4. 체크포인트 확인
print(f"\n[4] 모델 체크포인트 확인")
print(f"   - Checkpoint Dir : {CHECKPOINT_DIR} [{'✅ Exists' if CHECKPOINT_DIR.exists() else '❌ Missing'}]")
print(f"   - Checkpoint File: {CHECKPOINT_PT} [{'✅ Exists' if CHECKPOINT_PT.exists() else '❌ Missing'}]")

print("\n" + "="*60 + "\n")

# 필수 경로가 없을 경우 경고 발생 (선택 사항)
if not video_path_obj.exists():
    print("🚨 WARNING: 원본 비디오 파일이 존재하지 않습니다!")
if not CHECKPOINT_PT.exists():
    print("🚨 WARNING: 모델 가중치 파일(sam3.pt)이 존재하지 않습니다!")

## kpt에서 Boundary Box 가지고 오기

In [6]:
import json
import numpy as np

with open(KPT_DIR / "000000.json", 'r') as f:
    kpt_data = json.load(f)

ori_bbox = kpt_data['instance_info'][0]['bbox'][0]
ori_bbox = np.array(ori_bbox,dtype=np.float32)

kpt_width, kpt_height = 1280, 720

rel_box = [[
    ori_bbox[0] / kpt_width,
    ori_bbox[1] / kpt_height,
    ori_bbox[2] / kpt_width,
    ori_bbox[3] / kpt_height
]]
rel_box = np.array(rel_box,dtype=np.float32)

print("Original BBox:", ori_bbox)
print("Relative BBox:", rel_box)

## SAM3 모델 빌드

In [7]:
from pathlib import Path
from sam3.model_builder import build_sam3_video_model

sam3_model = build_sam3_video_model(checkpoint_path=CHECKPOINT_PT)  # SAM3 비디오 모델 빌드
predictor = sam3_model.tracker                                      # SAM3 비디오 예측기 초기화
predictor.backbone = sam3_model.detector.backbone                   # 백본 설정

print("SAM3 비디오 모델과 예측기가 성공적으로 초기화되었습니다.")

In [8]:
inference_state = predictor.init_state(video_path=VIDEO_PTH)    # 비디오 추론 상태 초기화
predictor.clear_all_points_in_video(inference_state)            # 비디오의 모든 포인트 지우기

In [9]:
!nvidia-smi

## 비디오 가지고 오기

In [ ]:
import cv2

# load the frames for visualization
cap = cv2.VideoCapture(VIDEO_PTH)       # 비디오 열기
video_frames_for_vis = []               #
while True:
    ret, frame = cap.read()
    if not ret:
        break
    video_frames_for_vis.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
cap.release()
frame0 = video_frames_for_vis[0]

width, height = frame0.shape[1], frame0.shape[0]

In [ ]:
import matplotlib.pyplot as plt
from sam3.visualization_utils import show_box, show_mask
ann_frame_idx = 0  # the frame index we interact with
ann_obj_id = 4  # give a unique id to each object we interact with (it can be any integers)

width = 1920
height = 1080

_, out_obj_ids, low_res_masks, video_res_masks  = predictor.add_new_points_or_box(
    inference_state=inference_state,
    frame_idx=ann_frame_idx,
    obj_id=ann_obj_id,
    box=rel_box,
)

box=np.array([[ 
    rel_box[0][0] * width,
    rel_box[0][1] * height,
    rel_box[0][2] * width,
    rel_box[0][3] * height
]], dtype=np.float32)


# # show the results on the current (interacted) frame
# plt.figure(figsize=(9, 6))
# plt.title(f"frame {ann_frame_idx}")
# plt.imshow(video_frames_for_vis[ann_frame_idx])
# show_box(box[0], plt.gca())
# show_mask((video_res_masks[0] > 0.0).cpu().numpy(), plt.gca(), obj_id=ann_obj_id)

## segmentation 추출

In [ ]:
import time

start_time = time.time()
# run propagation throughout the video and collect the results in a dict
video_segments = {}  # video_segments contains the per-frame segmentation results
for frame_idx, obj_ids, low_res_masks, video_res_masks, obj_scores in predictor.propagate_in_video(inference_state, start_frame_idx=0, max_frame_num_to_track=N_FRAMES, reverse=False, propagate_preflight=True):
    video_segments[frame_idx] = {
        out_obj_id: (video_res_masks[i] > 0.0).cpu().numpy()
        for i, out_obj_id in enumerate(out_obj_ids)
    }

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Propagation completed in {elapsed_time:.2f} seconds for {N_FRAMES} frames.")

### JSON 저장

In [ ]:
import json
import numpy as np

SAVE_DIR = OUTPUT_PATH / COMMON_PATH
SAVE_DIR.mkdir(parents=True, exist_ok=True)
JSON_SAVE_PATH = SAVE_DIR / "video_segments.json"

def mask_to_bbox(mask):
    """Convert a binary mask to bounding box [x_min, y_min, x_max, y_max]."""
    # 💡 [수정] 차원이 3차원(1, H, W)이면 2차원(H, W)으로 압축
    if mask.ndim == 3:
        mask = mask.squeeze()
        
    ys, xs = np.where(mask)
    if len(xs) == 0 or len(ys) == 0:
        return [0, 0, 0, 0]  # No mask found
    
    x_min, x_max = xs.min(), xs.max()
    y_min, y_max = ys.min(), ys.max()
    
    # JSON 저장을 위해 numpy int를 python int로 변환
    return [int(x_min), int(y_min), int(x_max), int(y_max)]

json_output = {}

# 진행 상황 표시를 위해 tqdm 사용 권장
from tqdm import tqdm

print("🔄 JSON 변환 및 저장 준비 중...")

for frame_idx, segments in tqdm(video_segments.items()):
    json_output[str(frame_idx)] = {} # JSON 키는 문자열이어야 안전함
    
    for obj_id, mask in segments.items():
        # 마스크 차원 정리 (여기서도 미리 해줘도 됨)
        if mask.ndim == 3:
            mask = mask.squeeze()
            
        bbox = mask_to_bbox(mask)
        
        json_output[str(frame_idx)][str(obj_id)] = {
            "bbox": bbox,
        }

try:
    print(f"💾 JSON 파일 저장 시작: {JSON_SAVE_PATH}")
    
    with open(JSON_SAVE_PATH, 'w') as f:
        json.dump(json_output, f) # indent=4를 빼면 용량이 줄어듭니다.
        
    print(f"✅ 저장 완료! : {JSON_SAVE_PATH}")
    
except Exception as e:
    print(f"❌ 저장 실패: {e}")

### 시각화

In [ ]:

# render the segmentation results every few frames
vis_frame_stride = 30
plt.close("all")
for out_frame_idx in range(0, len(video_frames_for_vis), vis_frame_stride):
    plt.figure(figsize=(6, 4))
    plt.title(f"frame {out_frame_idx}")
    plt.imshow(video_frames_for_vis[out_frame_idx])
    for out_obj_id, out_mask in video_segments[out_frame_idx].items():
        show_mask(out_mask, plt.gca(), obj_id=out_obj_id)

# Long Video

## 경로 설정

In [ ]:
from pathlib import Path
import pandas as pd

# Path 정리
DATA_DIR = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/data")
BASE_DIR = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/ASAN_01_mini_sam3/")
CSV_PATH = DATA_DIR / "metadata.csv"

# CSV 불러오기
df = pd.read_csv(CSV_PATH)

target = 0                             # 원하는 행 인덱스 설정
COMMON_PATH = df.loc[target,"common_path"]   # COMMON_PATH 추출

VIDEO_PTH = df.loc[target,"video_path"]
N_FRAMES = df.loc[target,"n_frames"]                    # 프레임 수 추출

FRAME_DIR = DATA_DIR / "1_FRAME" / COMMON_PATH          # 프레임 디렉토리 경로 설정
KPT_DIR = DATA_DIR / "2_KEYPOINTS" / COMMON_PATH        # 키포인트 디렉토리 경로 설정
MP4_PATH = DATA_DIR / "3_MP4" / f"{COMMON_PATH}.mp4"     # MP4 디렉토리 경로 설정    
INTERP_DIR = DATA_DIR / "4_INTERP_DATA" /COMMON_PATH    # 보간 데이터 디렉토리 경로 설정
OUTPUT_PATH = DATA_DIR / "test"

CHECKPOINT_DIR = DATA_DIR / "checkpoints/SAM3"
CHECKPOINT_PT = CHECKPOINT_DIR / "sam3.pt"

print(COMMON_PATH)

### 디버깅

In [ ]:
import sys

print("\n" + "="*60)
print("🐞 DEBUGGING REPORT")
print("="*60)

# 1. 기본 경로 및 CSV 확인
print(f"\n[1] 기본 경로 설정 확인")
print(f"   - DATA_DIR: {DATA_DIR} [{'✅ Exists' if DATA_DIR.exists() else '❌ Missing'}]")
print(f"   - BASE_DIR: {BASE_DIR} [{'✅ Exists' if BASE_DIR.exists() else '❌ Missing'}]")
print(f"   - CSV_PATH: {CSV_PATH} [{'✅ Exists' if CSV_PATH.exists() else '❌ Missing'}]")

# 2. 타겟 데이터 정보 (DataFrame 추출 값)
print(f"\n[2] Target Data Info (Index: {target})")
print(f"   - COMMON_PATH: {COMMON_PATH}")
print(f"   - N_FRAMES   : {N_FRAMES} (Type: {type(N_FRAMES).__name__})")
print(f"   - VIDEO_PTH (Raw): {VIDEO_PTH}")

# 3. 주요 디렉토리 및 파일 존재 여부 검증
# VIDEO_PTH는 문자열일 수 있으므로 Path 객체로 변환하여 확인
video_path_obj = Path(VIDEO_PTH)
print(f"\n[3] 주요 경로 유효성 검사")
print(f"   - Video File   : {video_path_obj} [{'✅ Exists' if video_path_obj.exists() else '❌ Missing'}]")
print(f"   - MP4_PATH     : {MP4_PATH} [{'✅ Exists' if MP4_PATH.exists() else '❌ Missing'}]")
print(f"   - FRAME_DIR    : {FRAME_DIR} [{'✅ Exists' if FRAME_DIR.exists() else '❌ Missing'}]")
print(f"   - KPT_DIR      : {KPT_DIR} [{'✅ Exists' if KPT_DIR.exists() else '❌ Missing'}]")
print(f"   - INTERP_DIR   : {INTERP_DIR} [{'✅ Exists' if INTERP_DIR.exists() else '❌ Missing'}]")
print(f"   - OUTPUT_PATH  : {OUTPUT_PATH} [{'✅ Exists' if OUTPUT_PATH.exists() else '⚠️ Will be created'}]")

# 4. 체크포인트 확인
print(f"\n[4] 모델 체크포인트 확인")
print(f"   - Checkpoint Dir : {CHECKPOINT_DIR} [{'✅ Exists' if CHECKPOINT_DIR.exists() else '❌ Missing'}]")
print(f"   - Checkpoint File: {CHECKPOINT_PT} [{'✅ Exists' if CHECKPOINT_PT.exists() else '❌ Missing'}]")

print("\n" + "="*60 + "\n")

# 필수 경로가 없을 경우 경고 발생 (선택 사항)
if not video_path_obj.exists():
    print("🚨 WARNING: 원본 비디오 파일이 존재하지 않습니다!")
if not CHECKPOINT_PT.exists():
    print("🚨 WARNING: 모델 가중치 파일(sam3.pt)이 존재하지 않습니다!")

## kpt에서 Boundary Box 가지고 오기

In [ ]:
import json
import numpy as np

with open(KPT_DIR / "000000.json", 'r') as f:
    kpt_data = json.load(f)

ori_bbox = kpt_data['instance_info'][0]['bbox'][0]
ori_bbox = np.array(ori_bbox,dtype=np.float32)

kpt_width, kpt_height = 1280, 720

rel_box = [[
    ori_bbox[0] / kpt_width,
    ori_bbox[1] / kpt_height,
    ori_bbox[2] / kpt_width,
    ori_bbox[3] / kpt_height
]]
rel_box = np.array(rel_box,dtype=np.float32)

print("Original BBox:", ori_bbox)
print("Relative BBox:", rel_box)

## SAM3 모델 빌드

In [ ]:
from pathlib import Path
from sam3.model_builder import build_sam3_video_model

sam3_model = build_sam3_video_model(checkpoint_path=CHECKPOINT_PT)  # SAM3 비디오 모델 빌드
predictor = sam3_model.tracker                                      # SAM3 비디오 예측기 초기화
predictor.backbone = sam3_model.detector.backbone                   # 백본 설정

print("SAM3 비디오 모델과 예측기가 성공적으로 초기화되었습니다.")

In [ ]:
inference_state = predictor.init_state(video_path=VIDEO_PTH)    # 비디오 추론 상태 초기화
predictor.clear_all_points_in_video(inference_state)            # 비디오의 모든 포인트 지우기

In [ ]:
!nvidia-smi

## 비디오 가지고 오기

In [ ]:
import cv2

# load the frames for visualization
cap = cv2.VideoCapture(VIDEO_PTH)       # 비디오 열기
video_frames_for_vis = []               #
while True:
    ret, frame = cap.read()
    if not ret:
        break
    video_frames_for_vis.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
cap.release()
frame0 = video_frames_for_vis[0]

width, height = frame0.shape[1], frame0.shape[0]

In [ ]:
import matplotlib.pyplot as plt
from sam3.visualization_utils import show_box, show_mask
ann_frame_idx = 0  # the frame index we interact with
ann_obj_id = 4  # give a unique id to each object we interact with (it can be any integers)

width = 1920
height = 1080

_, out_obj_ids, low_res_masks, video_res_masks  = predictor.add_new_points_or_box(
    inference_state=inference_state,
    frame_idx=ann_frame_idx,
    obj_id=ann_obj_id,
    box=rel_box,
)

box=np.array([[ 
    rel_box[0][0] * width,
    rel_box[0][1] * height,
    rel_box[0][2] * width,
    rel_box[0][3] * height
]], dtype=np.float32)


# # show the results on the current (interacted) frame
# plt.figure(figsize=(9, 6))
# plt.title(f"frame {ann_frame_idx}")
# plt.imshow(video_frames_for_vis[ann_frame_idx])
# show_box(box[0], plt.gca())
# show_mask((video_res_masks[0] > 0.0).cpu().numpy(), plt.gca(), obj_id=ann_obj_id)

## segmentation 추출

In [ ]:
import time

start_time = time.time()
# run propagation throughout the video and collect the results in a dict
video_segments = {}  # video_segments contains the per-frame segmentation results
for frame_idx, obj_ids, low_res_masks, video_res_masks, obj_scores in predictor.propagate_in_video(inference_state, start_frame_idx=0, max_frame_num_to_track=N_FRAMES, reverse=False, propagate_preflight=True):
    video_segments[frame_idx] = {
        out_obj_id: (video_res_masks[i] > 0.0).cpu().numpy()
        for i, out_obj_id in enumerate(out_obj_ids)
    }

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Propagation completed in {elapsed_time:.2f} seconds for {N_FRAMES} frames.")

### JSON 저장

In [11]:
from pathlib import Path
import pandas as pd
import json
import time
from tqdm import tqdm
import numpy as np
from sam3.model_builder import build_sam3_video_model
import cv2
import sys
sys.path.append('/workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/ASAN_01_mini_SAM3')
from func.mask_to_bbox import mask_to_bbox

# -----------------------------------------------------------------------------
# 1. 설정 및 데이터 로드 (Configuration & Data Loading)
# -----------------------------------------------------------------------------

# Path 정리
DATA_DIR = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/data")
BASE_DIR = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/ASAN_01_mini_SAM3/")
CSV_PATH = DATA_DIR / "metadata.csv"
OUTPUT_PATH = DATA_DIR / "test"
CHECKPOINT_DIR = DATA_DIR / "checkpoints/SAM3"
CHECKPOINT_PT = CHECKPOINT_DIR / "sam3.pt"

# CSV 불러오기 및 타겟 설정
df = pd.read_csv(CSV_PATH)

# 처리할 청크 사이즈 설정 (600 프레임)
CHUNK_SIZE = 600

for target in range(0,1):
    start_time = time.time()

    # 데이터 추출
    COMMON_PATH = df.loc[target, "common_path"]             # COMMON_PATH 추출
    VIDEO_PTH = df.loc[target, "video_path"]
    N_FRAMES = df.loc[target, "n_frames"]                   # 프레임 수 추출

    print(f"segment video : {COMMON_PATH} (Total Frames: {N_FRAMES})")

    # 세부 경로 설정
    FRAME_DIR = DATA_DIR / "1_FRAME" / COMMON_PATH          # 프레임 디렉토리 경로 설정
    KPT_DIR = DATA_DIR / "2_KEYPOINTS" / COMMON_PATH        # 키포인트 디렉토리 경로 설정
    MP4_PATH = DATA_DIR / "3_MP4" / f"{COMMON_PATH}.mp4"    # MP4 디렉토리 경로 설정    
    INTERP_DIR = DATA_DIR / "4_INTERP_DATA" / COMMON_PATH   # 보간 데이터 디렉토리 경로 설정
    SAVE_DIR = OUTPUT_PATH / COMMON_PATH                    # 저장 경로 설정
    SAVE_DIR.mkdir(parents=True, exist_ok=True)
    JSON_SAVE_PATH = SAVE_DIR / "video_segments.json"
    TIME_LOG_PATH = SAVE_DIR / "elapsed_time.txt"  # 시간 기록을 저장할 파일 경로 설정

    # -------------------------------------------------------------------------
    # 해상도 확인 (Video Resolution Check) - bbox 계산 전에 확인
    # -------------------------------------------------------------------------
    cap = cv2.VideoCapture(str(VIDEO_PTH))
    if cap.isOpened():
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        cap.release()
        print(f"🎥 Detected Video Resolution: {width}x{height}")
    else:
        print(f"⚠️ Warning: Could not open video at {VIDEO_PTH}. Using default 1920x1080.")
        width = 1920
        height = 1080

    # Keypoint 데이터 로드 및 BBox 계산
    with open(KPT_DIR / "000000.json", 'r') as f:
        kpt_data = json.load(f)

    ori_bbox = kpt_data['instance_info'][0]['bbox'][0]
    ori_bbox = np.array(ori_bbox, dtype=np.float32)

    kpt_width, kpt_height = 1280, 720

    rel_box = [[
        ori_bbox[0] / kpt_width,
        ori_bbox[1] / kpt_height,
        ori_bbox[2] / kpt_width,
        ori_bbox[3] / kpt_height
    ]]
    rel_box = np.array(rel_box, dtype=np.float32)

    # -----------------------------------------------------------------------------
    # 2. 모델 초기화 (Model Initialization)
    # -----------------------------------------------------------------------------
    sam3_model = build_sam3_video_model(checkpoint_path=CHECKPOINT_PT)  # SAM3 비디오 모델 빌드
    predictor = sam3_model.tracker                                      # SAM3 비디오 예측기 초기화
    predictor.backbone = sam3_model.detector.backbone                   # 백본 설정

    print("SAM3 비디오 모델과 예측기가 성공적으로 초기화되었습니다.")

    # -----------------------------------------------------------------------------
    # 3. 추론 준비 및 프롬프트 입력 (Inference Setup & Prompting)
    # -----------------------------------------------------------------------------
    inference_state = predictor.init_state(video_path=VIDEO_PTH)    # 비디오 추론 상태 초기화
    predictor.clear_all_points_in_video(inference_state)            # 비디오의 모든 포인트 지우기

    ann_frame_idx = 0   # the frame index we interact with
    ann_obj_id = 4      # give a unique id to each object we interact with

    # 초기 포인트/박스 추가
    _, out_obj_ids, low_res_masks, video_res_masks = predictor.add_new_points_or_box(
        inference_state=inference_state,
        frame_idx=ann_frame_idx,
        obj_id=ann_obj_id,
        box=rel_box,
    )

    # (참고용) 실제 픽셀 좌표 Box 계산
    box = np.array([[ 
        rel_box[0][0] * width,
        rel_box[0][1] * height,
        rel_box[0][2] * width,
        rel_box[0][3] * height
    ]], dtype=np.float32)

    # -----------------------------------------------------------------------------
    # 4. 비디오 전파 및 결과 수집 (Video Propagation with Chunks)
    # -----------------------------------------------------------------------------
    video_segments = {}  # 전체 결과를 모을 딕셔너리
    
    # 💡 [핵심 수정] 600 프레임 단위로 나누어 루프 실행
    # range(시작, 끝, 간격) -> 0, 600, 1200, ...
    for start_idx in range(0, N_FRAMES, CHUNK_SIZE):
        
        # 이번 턴에 처리할 프레임 수 계산 (마지막 청크는 600보다 작을 수 있음)
        frames_to_track = min(CHUNK_SIZE, N_FRAMES - start_idx)
        
        print(f"🔄 Processing Chunk: Frame {start_idx} ~ {start_idx + frames_to_track} (Size: {frames_to_track})")

        # propagate_in_video 호출
        # start_frame_idx를 청크의 시작점으로 설정
        # max_frame_num_to_track을 이번 청크 크기로 설정
        for frame_idx, obj_ids, low_res_masks, video_res_masks, obj_scores in predictor.propagate_in_video(
            inference_state,
            start_frame_idx=start_idx,
            max_frame_num_to_track=frames_to_track,
            reverse=False,
            propagate_preflight=True
        ):
            video_segments[frame_idx] = {
                out_obj_id: (video_res_masks[i] > 0.0).cpu().numpy()
                for i, out_obj_id in enumerate(out_obj_ids)
            }
        
        # (선택 사항) 필요하다면 여기서 메모리 정리(GC)를 수행할 수 있으나, 
        # SAM의 Tracking 연속성을 위해 inference_state는 유지해야 합니다.

    # -----------------------------------------------------------------------------
    # 5. 결과 변환 및 저장 (Result Processing & Saving)
    # -----------------------------------------------------------------------------
    print("🔄 JSON 변환 및 저장 준비 중...")

    json_output = {}

    for frame_idx, segments in tqdm(video_segments.items()):
        json_output[str(frame_idx)] = {} # JSON 키는 문자열이어야 안전함
        
        for obj_id, mask in segments.items():
            # 마스크 차원 정리
            if mask.ndim == 3:
                mask = mask.squeeze()
                
            bbox = mask_to_bbox(mask)
            
            json_output[str(frame_idx)][str(obj_id)] = {
                "bbox": bbox,
            }

    try:
        print(f"💾 JSON 파일 저장 시작: {JSON_SAVE_PATH}")
        
        with open(JSON_SAVE_PATH, 'w') as f:
            json.dump(json_output, f) # indent=4를 빼면 용량이 줄어듭니다.
            
        print(f"✅ 저장 완료! : {JSON_SAVE_PATH}")
        
    except Exception as e:
        print(f"❌ 저장 실패: {e}")

    # -----------------------------------------------------------------------------
    # 7. 실행 시간 별도 저장 (Save Elapsed Time)
    # -----------------------------------------------------------------------------
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"총 걸린 시간 = {elapsed_time:.2f} seconds")

    try:
        with open(TIME_LOG_PATH, 'w') as f:
            f.write(f"Total execution time: {elapsed_time:.2f} seconds\n")  # 시간을 소수점 둘째 자리까지 기록
            f.write(f"Processed Frames: {N_FRAMES}\n")
            f.write(f"Resolution: {width}x{height}\n")
            f.write(f"Chunk Size: {CHUNK_SIZE}\n") # 청크 사이즈도 기록
            
        print(f"⏱️ 실행 시간 기록 저장 완료: {TIME_LOG_PATH}") 

    except Exception as e:
        print(f"❌ 시간 기록 저장 실패: {e}")

In [14]:
from pathlib import Path
import subprocess

# -------------------------------------------------------
# 🎯 사용자 지정: 대상 영상 이름 (확장자 제외)
# -------------------------------------------------------
target_video = "M06_VISIT2_UpperLowerLimb"

# -------------------------------------------------------
# 🧭 경로 설정
# -------------------------------------------------------
BASE_DIR = Path("/workspace/nas203/ds_RehabilitationMedicineData/data/d02/Won_Kim_research_at_Bosanjin/M06")
OutputRoot  = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/Won_Kim_research_at_Bosanjin/M06")

# 대상 영상 탐색
video_candidates = list(BASE_DIR.glob(f"{target_video}.*")) 
if not video_candidates:
    raise FileNotFoundError(f"❌ 대상 영상이 없습니다: {target_video}")

Input = video_candidates[0]
OutputDir = OutputRoot / target_video
OutputDir.mkdir(parents=True, exist_ok=True)

print(f"""
📁 디렉토리 구조 설정 완료:
   • Input Video : {Input}
   • OutputDir   : {OutputDir}
""")

# -------------------------------------------------------
# ✂️ 세그먼트 정의 (frame 단위)
# -------------------------------------------------------
segments = [
    (0*30,    35*30,   "frontal__pec_dec_fly__1"),        # 0초 ~ 35초
    (52*30,   100*30,  "frontal__pec_dec_fly__2"),        # 52초 ~ 100초
    (190*30,  235*30,  "frontal__shoulder_press__1"),     # 190초 ~ 235초
    (250*30,  290*30,  "frontal__shoulder_press__2"),     # 250초 ~ 290초
    (310*30,  347*30,  "frontal__shoulder_flexion__1"),   # 310초 ~ 347초
    (348*30,  370*30,  "frontal__shoulder_flexion__2"),   # 348초 ~ 370초
    (375*30,  400*30,  "frontal__longsitting_knee_flexion__1"), # 375초 ~ 400초
    (425*30,  470*30,  "frontal__shoulder_flexion__3"),   # 425초 ~ 470초
    (530*30,  560*30,  "frontal__rowing__1"),             # 530초 ~ 560초
    (590*30,  605*30,  "frontal__rowing__2"),             # 590초 ~ 605초
    (830*30,  975*30,  "lateral__longsitting_knee_flexion__2"), # 830초 ~ 975초
    (975*30,  1025*30, "lateral__longsitting_knee_flexion__3"), # 975초 ~ 1025초
    (1040*30, 1100*30, "lateral__longsitting_knee_flexion__4"), # 1040초 ~ 1100초
    (1120*30, 1165*30, "lateral__slr__1"),                # 1120초 ~ 1165초
    (1170*30, 1225*30, "lateral__slr__2"),                # 1170초 ~ 1225초
    (1240*30, 1290*30, "lateral__slr__3"),                # 1240초 ~ 1290초
    (1300*30, 1350*30, "lateral__slr__4"),                # 1300초 ~ 1350초
]

# -------------------------------------------------------
# 💾 segmentation txt 저장
# -------------------------------------------------------
segment_txt = OutputDir / "segment_frames.txt"
with open(segment_txt, "w", encoding="utf-8") as f:
    f.write("# label, start_frame, end_frame\n")
    for start, end, label in segments:
        f.write(f"{label}, {start}, {end}\n")

print(f"📝 segmentation info saved: {segment_txt}")

# -------------------------------------------------------
# ✂️ FFmpeg 컷팅 함수
# -------------------------------------------------------
def cut_video(input_video, start_frame, end_frame, output_video, fps=30):
    """FFmpeg으로 mp4 구간 자르기 (frame 기반, 정확/빠름/디버깅 포함)"""
    def to_ffmpeg_time(frame_idx):
        t = frame_idx / fps
        m, s = divmod(t, 60)
        return f"{int(m):02d}:{int(s):02d}"

    start = to_ffmpeg_time(start_frame)
    end = to_ffmpeg_time(end_frame)

    cmd = [
        "ffmpeg",
        "-y",
        "-ss", start,
        "-to", end,
        "-i", str(input_video),
        "-c", "copy",
        str(output_video)
    ]

    print(f"[DEBUG] 실행 명령어: {' '.join(cmd)}")
    proc = subprocess.run(cmd, capture_output=True, text=True)

    if proc.returncode != 0:
        print(f"❌ ffmpeg 오류:\n{proc.stderr}")
    else:
        print(f"✅ ffmpeg 성공적으로 실행됨")

    if Path(output_video).exists():
        size = Path(output_video).stat().st_size / 1024
        print(f"📁 파일 생성 확인: {output_video} ({size:.1f} KB)")
    else:
        print(f"⚠️ 파일이 생성되지 않았습니다: {output_video}")

# -------------------------------------------------------
# 🚀 컷팅 실행 (OutputDir에 직접 저장)
# -------------------------------------------------------
for start, end, label in segments:
    output_path = OutputDir / f"{label}.mp4"
    cut_video(Input, start, end, output_path)
